In [22]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
pd.set_option("display.max_columns", 0)

In [2]:
secrets_file = open("secrets.txt","r")

In [3]:
string = secrets_file.read()

In [4]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [5]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [6]:
# we will use our 'first paid music' playlist as an example:
playlist = sp.user_playlist_tracks("spotify", "5xqpyfZyS1DVysoevdVyEn")

In [7]:
# playlist

{'href': 'https://api.spotify.com/v1/playlists/5xqpyfZyS1DVysoevdVyEn/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2016-04-13T17:09:41Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/secretagentfrog'},
    'href': 'https://api.spotify.com/v1/users/secretagentfrog',
    'id': 'secretagentfrog',
    'type': 'user',
    'uri': 'spotify:user:secretagentfrog'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1xU878Z1QtBldR7ru9owdU'},
       'href': 'https://api.spotify.com/v1/artists/1xU878Z1QtBldR7ru9owdU',
       'id': '1xU878Z1QtBldR7ru9owdU',
       'name': 'Phoenix',
       'type': 'artist',
       'uri': 'spotify:artist:1xU878Z1QtBldR7ru9owdU'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
   

In [14]:
playlist['total']

10991

In [8]:
# Let's look at items and total:
playlist.keys() 

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [13]:
# playlist['items'] contains the tracks on the playlist
# playlist["items"][0]['track']

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1xU878Z1QtBldR7ru9owdU'},
    'href': 'https://api.spotify.com/v1/artists/1xU878Z1QtBldR7ru9owdU',
    'id': '1xU878Z1QtBldR7ru9owdU',
    'name': 'Phoenix',
    'type': 'artist',
    'uri': 'spotify:artist:1xU878Z1QtBldR7ru9owdU'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'US',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',
   'OM',
   '

In [15]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) # respectful nap
    return tracks

In [16]:
all_tracks = get_playlist_tracks("5xqpyfZyS1DVysoevdVyEn")
len(all_tracks)

10991

In [24]:
from pandas import json_normalize
tracks2 = json_normalize(all_tracks)

In [25]:
tracks2.head()

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls.spotify,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url
0,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/5MBUL8d9FAYCCMr...,https://api.spotify.com/v1/albums/5MBUL8d9FAYC...,5MBUL8d9FAYCCMrogOFX68,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",United,2000-03-06,day,10,album,spotify:album:5MBUL8d9FAYCCMrogOFX68,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,266600,False,False,FRS639900098,https://open.spotify.com/track/3AA8xNhDC0Mpqwk...,https://api.spotify.com/v1/tracks/3AA8xNhDC0Mp...,3AA8xNhDC0MpqwkGX3EP5V,False,If I Ever Feel Better,66,https://p.scdn.co/mp3-preview/eb1e1d7972172e23...,True,4,track,spotify:track:3AA8xNhDC0MpqwkGX3EP5V,None
1,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,single,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/6Maw9mmnklMBmaL...,https://api.spotify.com/v1/albums/6Maw9mmnklMB...,6Maw9mmnklMBmaLrl1W6v7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Dance To The Underground,2001,year,4,album,spotify:album:6Maw9mmnklMBmaLrl1W6v7,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,243213,False,False,US2B30405301,https://open.spotify.com/track/2i8lRhmZNKaM1yp...,https://api.spotify.com/v1/tracks/2i8lRhmZNKaM...,2i8lRhmZNKaM1ypHwRybuD,False,Dance To The Underground,29,https://p.scdn.co/mp3-preview/bf9953e29d806a33...,True,1,track,spotify:track:2i8lRhmZNKaM1ypHwRybuD,None
2,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/6IvwFVN9YI9Bokk...,https://api.spotify.com/v1/albums/6IvwFVN9YI9B...,6IvwFVN9YI9Bokka32c80b,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",UltraSex,2002,year,13,album,spotify:album:6IvwFVN9YI9Bokka32c80b,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,234000,False,False,US25T9915124,https://open.spotify.com/track/4flxgPC0426CEeS...,https://api.spotify.com/v1/tracks/4flxgPC0426C...,4flxgPC0426CEeSrzQIic0,False,How We Do,34,https://p.scdn.co/mp3-preview/7b8c0e7b8498eaf4...,True,1,track,spotify:track:4flxgPC0426CEeSrzQIic0,None
3,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/5sgLqGnrMemtvQ5...,https://api.spotify.com/v1/albums/5sgLqGnrMemt...,5sgLqGnrMemtvQ5c4GUhab,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",The Teaches of Peaches,2002-10-08,day,17,album,spotify:album:5sgLqGnrMemtvQ5c4GUhab,[{'extern

In [34]:
def expand_list_dict2(row):
    df = json_normalize(row['track.artists'])
    df['song_id'] = row['track.id']
    return df


tracks2['artists_dfs'] = tracks2.apply(expand_list_dict2, axis=1)

In [37]:
tracks2['artists_dfs'][0]

,href,id,name,type,uri,external_urls.spotify,song_id
0,https://api.spotify.com/v1/artists/1xU878Z1QtB...,1xU878Z1QtBldR7ru9owdU,Phoenix,artist,spotify:artist:1xU878Z1QtBldR7ru9owdU,https://open.spotify.com/artist/1xU878Z1QtBldR...,3AA8xNhDC0MpqwkGX3EP5V


In [41]:
# now we create a new dataframe with all these artists
artist_df2 = pd.DataFrame(columns=['href', 'id', 'name', 'type', 'uri'])
for mini_df in tracks2['artists_dfs']:
    #display(val)
    artist_df2 = pd.concat([artist_df2, mini_df], axis=0)
    
artist_df2

,href,id,name,type,uri,external_urls.spotify,song_id
0,https://api.spotify.com/v1/artists/1xU878Z1QtB...,1xU878Z1QtBldR7ru9owdU,Phoenix,artist,spotify:artist:1xU878Z1QtBldR7ru9owdU,https://open.spotify.com/artist/1xU878Z1QtBldR...,3AA8xNhDC0MpqwkGX3EP5V
0,https://api.spotify.com/v1/artists/2oWyxHNcxMc...,2oWyxHNcxMcrvNIHMX2apR,Radio 4,artist,spotify:artist:2oWyxHNcxMcrvNIHMX2apR,https://open.spotify.com/artist/2oWyxHNcxMcrvN...,2i8lRhmZNKaM1ypHwRybuD
0,https://api.spotify.com/v1/artists/2UyhhfLOvfL...,2UyhhfLOvfLs7ZhzsAaNC3,Mount Sims,artist,spotify:artist:2UyhhfLOvfLs7ZhzsAaNC3,https://open.spotify.com/artist/2UyhhfLOvfLs7Z...,4flxgPC0426CEeSrzQIic0
0,https://api.spotify.com/v1/artists/1gkSl4XpHIH...,1gkSl4XpHIHI4I1WQbfXOE,Peaches,artist,spotify:artist:1gkSl4XpHIHI4I1WQbfXOE,https://open.spotify.com/artist/1gkSl4XpHIHI4I...,1XHFob24QklIXtLRopKirJ
0,https://api.spotify.com/v1/artists/3GpAL7oEFD3...,3GpAL7oEFD37IJDOOiirqy,Zongamin,artist,spotify:artist:3GpAL7oEFD37IJDOOiirqy,https://open.spotify.com/artist/3GpAL7oEFD37IJ...,4JB847zlgViLq8tJIzRsZy
...,...,...,...,...,...,...,...
0,https://api.spotify.com/v1/artists/4HpHDJqNFP3...,4HpHDJqNFP3XsGRy0SKTq4,SLEEP RADIO,artist,spotify:artist:4HpHDJqNFP3XsGRy0SKTq4,https://open.spotify.com/artist/4HpHDJqNFP3XsG...,1v6vHkDHBjm4OHUzeOKber
0,https://api.spotify.com/v1/artists/2sriRQRt36D...,2sriRQRt36DnfHtD68zZlj,Dorian Concept,artist,spotify:artist:2sriRQRt36DnfHtD68zZlj,https://open.spotify.com/artist/2sriRQRt36DnfH...,5TN8n4LQS8Or2WEnU56pXs
0,https://api.spotify.com/v1/artists/54QMjE4toDf...,54QMjE4toDfiCryzYWCpXX,Metronomy,artist,spotify:artist:54QMjE4toDfiCryzYWCpXX,https://open.spotify.com/artist/54QMjE4toDfiCr...,6MEzr1cmAJi3FXBjXVXMnm
0,https://api.spotify.com/v1/artists/5wFXmYsg3KF...,5wFXmYsg3KFJ8BDsQudJ4f,Manchester Orchestra,artist,spotify:artist:5wFXmYsg3KFJ8BDsQudJ4f,https://open.spotify.com/artist/5wFXmYsg3KFJ8B...,6ovSjbBKatyx9KvWsxkAZo


In [42]:
# now we merge (join) the two dataframes, keeping only the fields we need
df_merged2 = pd.merge(left=tracks2,
                    right=artist_df2,
                    how='inner',
                    left_on='track.id',
                    right_on='song_id')
df_merged2.head()

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls.spotify,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url,artists_dfs,href,id,name,type,uri,external_urls.spotify,song_id
0,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/5MBUL8d9FAYCCMr...,https://api.spotify.com/v1/albums/5MBUL8d9FAYC...,5MBUL8d9FAYCCMrogOFX68,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",United,2000-03-06,day,10,album,spotify:album:5MBUL8d9FAYCCMrogOFX68,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,266600,False,False,FRS639900098,https://open.spotify.com/track/3AA8xNhDC0Mpqwk...,https://api.spotify.com/v1/tracks/3AA8xNhDC0Mp...,3AA8xNhDC0MpqwkGX3EP5V,False,If I Ever Feel Better,66,https://p.scdn.co/mp3-preview/eb1e1d7972172e23...,True,4,track,spotify:track:3AA8xNhDC0MpqwkGX3EP5V,None,...,https://api.spotify.com/v1/artists/1xU878Z1QtB...,1xU878Z1QtBldR7ru9owdU,Phoenix,artist,spotify:artist:1xU878Z1QtBldR7ru9owdU,https://open.spotify.com/artist/1xU878Z1QtBldR...,3AA8xNhDC0MpqwkGX3EP5V
1,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,single,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/6Maw9mmnklMBmaL...,https://api.spotify.com/v1/albums/6Maw9mmnklMB...,6Maw9mmnklMBmaLrl1W6v7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Dance To The Underground,2001,year,4,album,spotify:album:6Maw9mmnklMBmaLrl1W6v7,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,243213,False,False,US2B30405301,https://open.spotify.com/track/2i8lRhmZNKaM1yp...,https://api.spotify.com/v1/tracks/2i8lRhmZNKaM...,2i8lRhmZNKaM1ypHwRybuD,False,Dance To The Underground,29,https://p.scdn.co/mp3-preview/bf9953e29d806a33...,True,1,track,spotify:track:2i8lRhmZNKaM1ypHwRybuD,None,...,https://api.spotify.com/v1/artists/2oWyxHNcxMc...,2oWyxHNcxMcrvNIHMX2apR,Radio 4,artist,spotify:artist:2oWyxHNcxMcrvNIHMX2apR,https://open.spotify.com/artist/2oWyxHNcxMcrvN...,2i8lRhmZNKaM1ypHwRybuD
2,2016-04-13T17:09:41Z,False,None,https://open.spotify.com/user/secretagentfrog,https://api.spotify.com/v1/users/secretagentfrog,secretagentfrog,user,spotify:user:secretagentfrog,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/6IvwFVN9YI9Bokk...,https://api.spotify.com/v1/albums/6IvwFVN9YI9B...,6IvwFVN9YI9Bokka32c80b,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",UltraSex,2002,year,13,album,spotify:album:6IvwFVN9YI9Bokka32c80b,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,234000,False,False,US25T9915124,https://open.spotify.com/track/4flxgPC0426CEeS...,https://api.spotify.com/v1/tracks/4flxgPC0426C...,4flxgPC0426CEeSrzQIic0,False,How We Do,34,https://p.scdn.co/mp3-preview/7b8c0e7b8498eaf4...,True,1,track,spotify:track:4flxgPC0426CEeSrzQIic0,None,...,https://api.spotify.com/v1/artis

In [43]:
df_final2 = df_merged2[['track.name', 'name', 'song_id']]
df_final2

,track.name,name,song_id
0,If I Ever Feel Better,Phoenix,3AA8xNhDC0MpqwkGX3EP5V
1,Dance To The Underground,Radio 4,2i8lRhmZNKaM1ypHwRybuD
2,How We Do,Mount Sims,4flxgPC0426CEeSrzQIic0
3,Fuck the Pain Away,Peaches,1XHFob24QklIXtLRopKirJ
4,Street Surgery 2,Zongamin,4JB847zlgViLq8tJIzRsZy
...,...,...,...
12676,Hate Is an Attractive Force,SLEEP RADIO,1v6vHkDHBjm4OHUzeOKber
12677,Promises,Dorian Concept,5TN8n4LQS8Or2WEnU56pXs
12678,16 Beat,Metronomy,6MEzr1cmAJi3FXBjXVXMnm
12679,Amplified In The Silence,Manchester Orchestra,6ovSjbBKatyx9KvWsxkAZo


In [44]:
chunks = [(i, i+100) for i in range(0, len(df_final2), 100)]
chunks
audio_features_list = []
for chunk in chunks:
    id_list100 = df_final2['song_id'][chunk[0]:chunk[1]]
    audio_features_list = audio_features_list + sp.audio_features(id_list100)
    sleep(randint(1,3000)/1000)
len(audio_features_list)

12681

In [45]:
audio_features_df = json_normalize(audio_features_list)

In [46]:
audio_features_df.drop_duplicates(inplace=True) 

In [47]:
df_w_audio_ft = pd.merge(left=df_final2,
                        right=audio_features_df,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,If I Ever Feel Better,Phoenix,3AA8xNhDC0MpqwkGX3EP5V,0.755,0.553,6,-9.449,0,0.1070,0.04920,0.006070,0.0646,0.836,119.700,audio_features,3AA8xNhDC0MpqwkGX3EP5V,spotify:track:3AA8xNhDC0MpqwkGX3EP5V,https://api.spotify.com/v1/tracks/3AA8xNhDC0Mp...,https://api.spotify.com/v1/audio-analysis/3AA8...,266600,4
1,Dance To The Underground,Radio 4,2i8lRhmZNKaM1ypHwRybuD,0.667,0.667,7,-6.267,1,0.0388,0.00013,0.003850,0.0711,0.843,130.321,audio_features,2i8lRhmZNKaM1ypHwRybuD,spotify:track:2i8lRhmZNKaM1ypHwRybuD,https://api.spotify.com/v1/tracks/2i8lRhmZNKaM...,https://api.spotify.com/v1/audio-analysis/2i8l...,243213,4
2,How We Do,Mount Sims,4flxgPC0426CEeSrzQIic0,0.826,0.601,10,-3.966,0,0.0515,0.19500,0.000003,0.0700,0.948,126.971,audio_features,4flxgPC0426CEeSrzQIic0,spotify:track:4flxgPC0426CEeSrzQIic0,https://api.spotify.com/v1/tracks/4flxgPC0426C...,https://api.spotify.com/v1/audio-analysis/4flx...,234000,4
3,Fuck the Pain Away,Peaches,1XHFob24QklIXtLRopKirJ,0.905,0.945,4,-2.630,1,0.1370,0.03540,0.005290,0.0707,0.944,131.597,audio_features,1XHFob24QklIXtLRopKirJ,spotify:track:1XHFob24QklIXtLRopKirJ,https://api.spotify.com/v1/tracks/1XHFob24QklI...,https://api.spotify.com/v1/audio-analysis/1XHF...,250693,4
4,Street Surgery 2,Zongamin,4JB847zlgViLq8tJIzRsZy,0.797,0.536,11,-8.267,0,0.2660,0.03150,0.000029,0.1700,0.704,102.881,audio_features,4JB847zlgViLq8tJIzRsZy,spotify:track:4JB847zlgViLq8tJIzRsZy,https://api.spotify.com/v1/tracks/4JB847zlgViL...,https://api.spotify.com/v1/audio-analysis/4JB8...,224800,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12676,Hate Is an Attractive Force,SLEEP RADIO,1v6vHkDHBjm4OHUzeOKber,0.759,0.121,7,-25.346,0,0.0284,0.41000,0.006100,0.0905,0.523,90.029,audio_features,1v6vHkDHBjm4OHUzeOKber,spotify:track:1v6vHkDHBjm4OHUzeOKber,https://api.spotify.com/v1/tracks/1v6vHkDHBjm4...,https://api.spotify.com/v1/audio-analysis/1v6v...,170667,4
12677,Promises,Dorian Concept,5TN8n4LQS8Or2WEnU56pXs,0.621,0.675,10,-9.347,0,0.0685,0.38500,0.033700,0.3470,0.323,124.056,audio_features,5TN8n4LQS8Or2WEnU56pXs,spotify:track:5TN8n4LQS8Or2WEnU56pXs,https://api.spotify.com/v1/tracks/5TN8n4LQS8Or...,https://api.spotify.com/v1/audio-analysis/5TN8...,237064,1
12678,16 Beat,Metronomy,6MEzr1cmAJi3FXBjXVXMnm,0.781,0.812,7,-11.866,1,0.0468,0.01860,0.586000,0.0716,0.785,115.001,audio_features,6MEzr1cmAJi3FXBjXVXMnm,spotify:track:6MEzr1cmAJi3FXBjXVXMnm,https://api.spotify.com/v1/tracks/6MEzr1cmAJi3...,https://api.spotify.com/v1/audio-analysis/6MEz...,196027,4
12679,Amplified In The Silence,Manchester Orchestra,6ovSjbBKatyx9KvWsxkAZo,0.557,0.117,1,-13.880,1,0.0344,0.95000,0.000269,0.1120,0.122,111.742,audio_features,6ovSjbBKatyx9KvWsxkAZo,spotify:track:6ovSjbBKatyx9KvWsxkAZo,https://api.spotify.com/v1/tracks/6ovSjbBKatyx...,https://api.spotify.com/v1/audio-analysis/6ovS...,223107,4


In [60]:
# define a function to get more playlist
def get_playlist_dataframe(sp, playlist_id):
    def expand_list_dict2(row):
        df = json_normalize(row['track.artists'])
        df['song_id'] = row['track.id']
        return df

    def fetch_audio_features(sp, id_list):
        chunks = [(i, i+100) for i in range(0, len(id_list), 100)]
        audio_features_list = []

        for chunk in chunks:
            id_list100 = id_list[chunk[0]:chunk[1]]

            # Filter out None or invalid track IDs
            id_list100 = [track_id for track_id in id_list100 if track_id is not None]

            if id_list100:
                audio_features_list += sp.audio_features(id_list100)
                sleep(randint(1, 3000) / 1000)
            else:
                # Append None for each track ID that is None
                audio_features_list += [None] * len(id_list100)

        return audio_features_list

    # Fetch all tracks from the playlist
    results = sp.user_playlist_tracks("spotify", playlist_id)
    tracks = results['items']

    while results['next'] is not None:
        results = sp.next(results)
        tracks += results['items']
        sleep(randint(1, 3000) / 1000)  # respectful nap

    # Create a DataFrame for the tracks
    tracks_df = json_normalize(tracks)

    # Create a DataFrame for the artists
    tracks_df['artists_dfs'] = tracks_df.apply(expand_list_dict2, axis=1)
    artist_df = pd.DataFrame(columns=['href', 'id', 'name', 'type', 'uri'])

    for mini_df in tracks_df['artists_dfs']:
        artist_df = pd.concat([artist_df, mini_df], axis=0)

    # Merge the two dataframes, keeping only the fields needed
    df_merged = pd.merge(left=tracks_df,
                        right=artist_df,
                        how='inner',
                        left_on='track.id',
                        right_on='song_id')

    # Select the final columns
    df_final = df_merged[['track.name', 'name', 'song_id']]

    # Fetch audio features for the tracks
    id_list = df_final['song_id'].tolist()
    audio_features_list = fetch_audio_features(sp, id_list)

    # Create a DataFrame for the audio features
    audio_features_df = pd.DataFrame(audio_features_list)

    # Drop duplicates in the audio features dataframe
    audio_features_df.drop_duplicates(inplace=True)

    # Merge the final dataframe with the audio features dataframe
    df_w_audio_ft = pd.merge(left=df_final,
                             right=audio_features_df,
                             how='inner',
                             left_on='song_id',
                             right_on='id')

    return df_w_audio_ft


In [51]:
# Usage
playlist_id_2 = "6FKDzNYZ8IW1pvYVF4zUN2"
df_playlist_2 = get_playlist_dataframe(sp, playlist_id_2)


In [54]:
df_playlist_2.head()

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Timber (feat. Ke$ha),Pitbull,3tgZ9vmhuAY9wEoNUJskzV,0.589,0.966,11,-4.106,1,0.0972,0.030000,0.00000,0.1470,0.802,129.983,audio_features,3tgZ9vmhuAY9wEoNUJskzV,spotify:track:3tgZ9vmhuAY9wEoNUJskzV,https://api.spotify.com/v1/tracks/3tgZ9vmhuAY9...,https://api.spotify.com/v1/audio-analysis/3tgZ...,204200,4
1,Timber (feat. Ke$ha),Kesha,3tgZ9vmhuAY9wEoNUJskzV,0.589,0.966,11,-4.106,1,0.0972,0.030000,0.00000,0.1470,0.802,129.983,audio_features,3tgZ9vmhuAY9wEoNUJskzV,spotify:track:3tgZ9vmhuAY9wEoNUJskzV,https://api.spotify.com/v1/tracks/3tgZ9vmhuAY9...,https://api.spotify.com/v1/audio-analysis/3tgZ...,204200,4
2,Starships,Nicki Minaj,2EBCVPNAG46nbgs6jXPGvv,0.747,0.716,11,-2.457,0,0.0750,0.135000,0.00000,0.2510,0.751,125.008,audio_features,2EBCVPNAG46nbgs6jXPGvv,spotify:track:2EBCVPNAG46nbgs6jXPGvv,https://api.spotify.com/v1/tracks/2EBCVPNAG46n...,https://api.spotify.com/v1/audio-analysis/2EBC...,210627,4
3,Wild Strawberries,PNAU,6pGUGTIaZ1H4jKHIL4Fged,0.647,0.933,7,-4.056,1,0.1110,0.000351,0.00277,0.3340,0.332,119.921,audio_features,6pGUGTIaZ1H4jKHIL4Fged,spotify:track:6pGUGTIaZ1H4jKHIL4Fged,https://api.spotify.com/v1/tracks/6pGUGTIaZ1H4...,https://api.spotify.com/v1/audio-analysis/6pGU...,235107,4
4,Papaoutai,Stromae,09TcIuH1ZO7i4vicWKoaN2,0.733,0.818,10,-7.222,0,0.0859,0.024100,0.00000,0.0636,0.253,116.019,audio_features,09TcIuH1ZO7i4vicWKoaN2,spotify:track:09TcIuH1ZO7i4vicWKoaN2,https://api.spotify.com/v1/tracks/09TcIuH1ZO7i...,https://api.spotify.com/v1/audio-analysis/09Tc...,232147,4


In [62]:
df = pd.concat([df_w_audio_ft,df_playlist_2], axis = 0)

In [67]:
df.drop_duplicates(inplace = True)

In [68]:
df

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,If I Ever Feel Better,Phoenix,3AA8xNhDC0MpqwkGX3EP5V,0.755,0.553,6,-9.449,0,0.1070,0.04920,0.006070,0.0646,0.8360,119.700,audio_features,3AA8xNhDC0MpqwkGX3EP5V,spotify:track:3AA8xNhDC0MpqwkGX3EP5V,https://api.spotify.com/v1/tracks/3AA8xNhDC0Mp...,https://api.spotify.com/v1/audio-analysis/3AA8...,266600,4
1,Dance To The Underground,Radio 4,2i8lRhmZNKaM1ypHwRybuD,0.667,0.667,7,-6.267,1,0.0388,0.00013,0.003850,0.0711,0.8430,130.321,audio_features,2i8lRhmZNKaM1ypHwRybuD,spotify:track:2i8lRhmZNKaM1ypHwRybuD,https://api.spotify.com/v1/tracks/2i8lRhmZNKaM...,https://api.spotify.com/v1/audio-analysis/2i8l...,243213,4
2,How We Do,Mount Sims,4flxgPC0426CEeSrzQIic0,0.826,0.601,10,-3.966,0,0.0515,0.19500,0.000003,0.0700,0.9480,126.971,audio_features,4flxgPC0426CEeSrzQIic0,spotify:track:4flxgPC0426CEeSrzQIic0,https://api.spotify.com/v1/tracks/4flxgPC0426C...,https://api.spotify.com/v1/audio-analysis/4flx...,234000,4
3,Fuck the Pain Away,Peaches,1XHFob24QklIXtLRopKirJ,0.905,0.945,4,-2.630,1,0.1370,0.03540,0.005290,0.0707,0.9440,131.597,audio_features,1XHFob24QklIXtLRopKirJ,spotify:track:1XHFob24QklIXtLRopKirJ,https://api.spotify.com/v1/tracks/1XHFob24QklI...,https://api.spotify.com/v1/audio-analysis/1XHF...,250693,4
4,Street Surgery 2,Zongamin,4JB847zlgViLq8tJIzRsZy,0.797,0.536,11,-8.267,0,0.2660,0.03150,0.000029,0.1700,0.7040,102.881,audio_features,4JB847zlgViLq8tJIzRsZy,spotify:track:4JB847zlgViLq8tJIzRsZy,https://api.spotify.com/v1/tracks/4JB847zlgViL...,https://api.spotify.com/v1/audio-analysis/4JB8...,224800,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12955,Dark Star - Homemade Weapons Remix,Homemade Weapons,28d1X9lfagOD4iFULH4qEK,0.337,0.977,9,-1.995,1,0.0976,0.00492,0.872000,0.2600,0.0582,169.867,audio_features,28d1X9lfagOD4iFULH4qEK,spotify:track:28d1X9lfagOD4iFULH4qEK,https://api.spotify.com/v1/tracks/28d1X9lfagOD...,https://api.spotify.com/v1/audio-analysis/28d1...,345706,4
12956,러시안 룰렛 Russian Roulette,Red Velvet,5HiSc2ZCGn8L3cH3qSwzBT,0.726,0.903,7,-2.277,0,0.0357,0.05500,0.000225,0.1580,0.9580,129.986,audio_features,5HiSc2ZCGn8L3cH3qSwzBT,spotify:track:5HiSc2ZCGn8L3cH3qSwzBT,https://api.spotify.com/v1/tracks/5HiSc2ZCGn8L...,https://api.spotify.com/v1/audio-analysis/5HiS...,211244,4
12957,Lovers on the Sun (feat. Sam Martin),David Guetta,2rgtCOVK2tpfao2VjURn84,0.645,0.891,6,-2.505,0,0.0387,0.09320,0.000004,0.3790,0.5680,124.915,audio_features,2rgtCOVK2tpfao2VjURn84,spotify:track:2rgtCOVK2tpfao2VjURn84,https://api.spotify.com/v1/tracks/2rgtCOVK2tpf...,https://api.spotify.com/v1/audio-analysis/2rgt...,203520,4
12958,Lovers on the Sun (feat. Sam Martin),Sam Martin,2rgtCOVK2tpfao2VjURn84,0.645,0.891,6,-2.505,0,0.0387,0.09320,0.000004,0.3790,0.5680,124.915,audio_features,2rgtCOVK2tpfao2VjURn84,spotify:track:2rgtCOVK2tpfao2VjURn84,https://api.spotify.com/v1/tracks/2rgtCOVK2tpf...,https://api.spotify.com/v1/audio-analysis/2rgt...,203520,4
